In [79]:
# Convert trip records into matrices

In [80]:
import h5py
import pandas as pd

In [81]:
from input_configuration import *
from scripts.EmmeProject import *
from scripts.skimming.SkimsAndPaths import *
sys.path.append(os.path.join(os.getcwd(),"inputs"))

In [82]:
# USER CLASSES DEFINED PER BELLEVUE'S 3 CLASSES
matrix_dict = json_to_dictionary("bkr_user_classes")

In [83]:
# Store in an h5 file
# all matrices must be stored in a "data" table to be read by emme
# trips.h5 -> data -> [sov,hov,transit]

## Load demand

In [84]:
# Read trip records
daysim_outputs_loc = r'R:\SoundCast\releases\TransportationFutures2010\outputs\daysim_outputs.h5'
my_store = h5py.File(daysim_outputs_loc, "r+")

In [85]:
#Read the Matrix File from the Dictionary File and Set Unique Matrix Names
matrix_dict = text_to_dictionary('demand_matrix_dictionary')
uniqueMatrices = set(matrix_dict.values())

In [86]:
#Stores in the HDF5 Container to read or write to
daysim_set = my_store['Trip']

# Select only trips departing between specified time period
# Between 5-6 pm
tod = 17

# Get the index for trips in TOD to only loop over those
deptm = np.asarray(daysim_set["deptm"])

df_deptm = pd.DataFrame(deptm/60,columns=['dephr'])
df_deptm['index_col'] = df_deptm.index

tod_index = df_deptm[df_deptm['dephr']==tod]['index_col'].as_matrix()

In [87]:
len(tod_index)

1114904

In [88]:
len(deptm)

15397776

In [89]:
851101/15397776

0

In [90]:
opcl = np.asarray(daysim_set["opcl"])
opcl = opcl.astype('int')


dpcl = np.asarray(daysim_set["dpcl"])
dpcl = dpcl.astype('int')


mode = np.asarray(daysim_set["mode"])
mode = mode.astype("int")

trexpfac = np.asarray(daysim_set["trexpfac"])

if not survey_seed_trips:
    vot = np.asarray(daysim_set["vot"])

dorp = np.asarray(daysim_set["dorp"])
dorp = dorp.astype('int')


toll_path = np.asarray(daysim_set["pathtype"])
toll_path = toll_path.astype('int')

my_store.close

<bound method File.close of <HDF5 file "daysim_outputs.h5" (mode r+)>>

**Create the BKR zone lookup for parcels**

In [91]:
# Using internal BKR zones:

In [92]:
bkr_lookup = pd.read_csv(r'D:/bkr/bkrtaz_internal_parcels.csv')

In [93]:
bkr_lookup = bkr_lookup[['TAZNUM','PARCELID']]

In [94]:
# Remove any duplicate rows, keeping the first occurance
# not sure why there are duplicates, but there are only ~ 50 out of 1176991
bkr_lookup.drop_duplicates(subset='PARCELID',keep='first',inplace=True)

# # Get zone numbers
bkr_zones = bkr_lookup.groupby('TAZNUM').count().index.tolist()

# add an extra zone to place the umatched parcels
##### Zone 
bkr_zones.append(bkr_zones[-1]+1)

dictZoneLookup = dict((value,index) for index,value in enumerate(bkr_zones))

zonesDim = len(dictZoneLookup)

**Join origin and destination parcel to BKR TAZ**

In [95]:
# Create otaz and dtaz arrays that contain BKR o and d taz definitions

In [96]:
opcl_df = pd.DataFrame(opcl,columns=['opcl'])
df = opcl_df.merge(bkr_lookup,left_on='opcl',right_on='PARCELID',how='left')

# Fill NaN rows with the fake BKR zone
df.fillna(bkr_zones[-1],inplace=True)

otaz = df['TAZNUM'].as_matrix()
del df, opcl_df

In [97]:
dpcl_df = pd.DataFrame(dpcl,columns=['dpcl'])
df = dpcl_df.merge(bkr_lookup,left_on='dpcl',right_on='PARCELID',how='left')

# Fill NaN rows with the fake BKR zone
df.fillna(bkr_zones[-1],inplace=True)

dtaz = df['TAZNUM'].as_matrix()
del df, dpcl_df

In [98]:
# Use daysim trips here
survey_seed_trips = False

#create & store in-memory numpy matrices in a dictionary. Key is matrix name, value is the matrix
#also load up the external and truck trips
demand_matrices={}

for matrix in list(uniqueMatrices):
    if matrix not in demand_matrices.keys():
        demand_matrix = np.zeros((bkr_zones[-1],bkr_zones[-1]), np.float16)
        demand_matrices.update({matrix : demand_matrix})
        
unprocessed = 0
processed = 0
# for x in range (0, len(otaz)):
for x in tod_index:

    processed += 1
    if vot[x] < 15: vot[x]=1
    elif vot[x] < 25: vot[x]=2
    else: vot[x]=3

    #get the matrix name from matrix_dict. Throw out school bus (8) for now.
    if mode[x] <= 0: 
         print x, mode[x]
    if mode[x]<8:
        #Only want drivers, transit trips.
        if dorp[x] <= 1:
            mat_name = matrix_dict[(int(mode[x]),int(vot[x]),int(toll_path[x]))]
            myOtaz = otaz[x]-1
            myDtaz = dtaz[x]-1

            demand_matrices[mat_name][myOtaz, myDtaz] = demand_matrices[mat_name][myOtaz, myDtaz] + 1

C:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\ipykernel\__main__.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [99]:
tod_index

array([       1,        9,       10, ..., 15397733, 15397741, 15397764], dtype=int64)

In [100]:
for x in tod_index[:10]:
    print x

1
9
10
15
30
35
37
89
117
118


In [101]:
demand_matrices['svnt1'].sum()

56032.0

In [102]:
######## Now we have matrices in memory
# svnt1 has infinite values...
# IGNORE svnt1 for now?
# Aggregate to fit BKR needs
sov =  demand_matrices['svnt1']+demand_matrices['svnt2']+demand_matrices['svnt3']+\
    demand_matrices['svtl1']+demand_matrices['svtl2']+demand_matrices['svtl3']

In [103]:
hov2 =  demand_matrices['h2nt1']+demand_matrices['h2nt2']+demand_matrices['h2nt3']+\
    demand_matrices['h2tl1']+demand_matrices['h2tl2']+demand_matrices['h2tl3']

In [104]:
hov3 =  demand_matrices['h3nt1']+demand_matrices['h3nt2']+demand_matrices['h3nt3']+\
    demand_matrices['h3tl1']+demand_matrices['h3tl2']+demand_matrices['h3tl3']

In [105]:
df = pd.DataFrame(sov)
# df.columns = dictZoneLookup.keys()
# df.index = dictZoneLookup.keys()
df.columns = range(1,bkr_zones[-1]+1)
df.index = range(1,bkr_zones[-1]+1)
df.to_csv('sov.csv')

# # totals by P&A
df = pd.DataFrame([df.sum(axis=0),df.sum(axis=1)]).T
df.columns = ['prd','attr']
df.to_csv('sov_p_a.csv',index_label='taz')

In [106]:
# df.sum()

In [107]:
df = pd.DataFrame(hov2)
df.columns = range(1,bkr_zones[-1]+1)
df.index = range(1,bkr_zones[-1]+1)
df.to_csv('hov2.csv')

df = pd.DataFrame([df.sum(axis=0),df.sum(axis=1)]).T
df.columns = ['prd','attr']
df.to_csv('hov2_p_a.csv',index_label='taz')

In [108]:
df = pd.DataFrame(hov3)
df.columns = range(1,bkr_zones[-1]+1)
df.index = range(1,bkr_zones[-1]+1)
df.to_csv('hov3.csv')

df = pd.DataFrame([df.sum(axis=0),df.sum(axis=1)]).T
df.columns = ['prd','attr']
df.to_csv('hov3_p_a.csv',index_label='taz')

In [109]:
# Get origin and destination totals to compare vs soundcast zones

In [110]:
# Total SOV Trips from this notebook
# 4304750

# compared to all-day soundcast run
# sov is mode 3
# daysim reports these as 
# 7076788

# still only getting about half of the trips

In [111]:
allmode = pd.DataFrame(mode)

In [112]:
len(allmode[allmode[0] == 3])

7076788

In [113]:
len(mode)

15397776